In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import math
import phe
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler
from phe import paillier
import bitstring
from bitstring import Bits
from secrets import token_hex
from Crypto.Cipher import DES

%matplotlib inline

In [105]:
def cholesky_decomposition(A):
    d = A.shape[0]
    L = A.copy()
    #L = np.zeros(A.shape)
    for i in range(d-1):
        for j in range(i+1,d):
            L[i][j] = 0.0
    for j in range(d):
        for k in range(j-1):
            for i in range(j,d):
                L[i][j] -= (L[i][k]*L[j][k])
        L[j][j] = math.sqrt(L[j][j])
        for k in range(j+1,d):
            L[k][j] /= L[j][j]
    return L

In [106]:
A = np.array([[ 6,  3,  4,  8],
       [ 3,  6,  5,  1],
       [ 4,  5, 10,  7],
       [ 8,  1,  7, 25]],dtype=np.float32)

In [107]:
L = cholesky_decomposition(A)

In [108]:
L

array([[2.4494898 , 0.        , 0.        , 0.        ],
       [1.2247448 , 2.4494898 , 0.        , 0.        ],
       [1.6329931 , 2.0412414 , 2.7080128 , 0.        ],
       [3.2659862 , 0.40824828, 0.6154576 , 3.7638633 ]], dtype=float32)

In [109]:
np.dot(L,L.T)

array([[ 6.0000005,  3.       ,  4.       ,  8.       ],
       [ 3.       ,  7.5      ,  7.       ,  4.9999995],
       [ 4.       ,  7.       , 14.166666 ,  7.8333335],
       [ 8.       ,  4.9999995,  7.8333335, 25.378788 ]], dtype=float32)

In [110]:
A = np.array([[ 6,  3,  4,  8],
       [ 3,  6,  5,  1],
       [ 4,  5, 10,  7],
       [ 8,  1,  7, 25]],dtype=np.float32)

In [111]:
L2 = np.linalg.cholesky(A)

In [112]:
L2

array([[ 2.4494898,  0.       ,  0.       ,  0.       ],
       [ 1.2247449,  2.1213202,  0.       ,  0.       ],
       [ 1.6329932,  1.4142135,  2.309401 ,  0.       ],
       [ 3.2659864, -1.4142135,  1.5877132,  3.132491 ]], dtype=float32)

In [67]:
np.dot(L2,L2.T)

array([[ 6.0000005,  3.0000002,  4.0000005,  8.000001 ],
       [ 3.0000002,  5.9999995,  5.       ,  1.0000007],
       [ 4.0000005,  5.       , 10.       ,  7.       ],
       [ 8.000001 ,  1.0000007,  7.       , 25.       ]], dtype=float32)

In [68]:
A

array([[ 6.,  3.,  4.,  8.],
       [ 3.,  6.,  5.,  1.],
       [ 4.,  5., 10.,  7.],
       [ 8.,  1.,  7., 25.]], dtype=float32)

In [69]:
def cholesky(A):
    """Performs a Cholesky decomposition of A, which must 
    be a symmetric and positive definite matrix. The function
    returns the lower variant triangular matrix, L."""
    n = len(A)

    # Create zero matrix for L
    L = np.zeros(A.shape)

    # Perform the Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                # LaTeX: l_{kk} = \sqrt{ a_{kk} - \sum^{k-1}_{j=1} l^2_{kj}}
                L[i][k] = np.sqrt(A[i][i] - tmp_sum)
            else:
                # LaTeX: l_{ik} = \frac{1}{l_{kk}} \left( a_{ik} - \sum^{k-1}_{j=1} l_{ij} l_{kj} \right)
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L

In [70]:
cholesky(A)

array([[ 2.44948974,  0.        ,  0.        ,  0.        ],
       [ 1.22474487,  2.12132034,  0.        ,  0.        ],
       [ 1.63299316,  1.41421356,  2.30940108,  0.        ],
       [ 3.26598632, -1.41421356,  1.58771324,  3.13249102]])

In [68]:
def square_root(x):
    e = [pow(4,i) for i in range(int(x)) if pow(4,i)<=x][-1]
    r = 0
    #print(e)
    while e != 0:
        if x >= r+e:
            x -= (r+e)
            r = (r >> 1) +e
        else:
            r = (r >> 1)
        e = (e >> 2)
    return r

In [69]:
square_root(100)

10

In [71]:
def poc_bitstring_class():
    binary = np.array(list(bin(255))[2:],dtype=np.bool)
    b = bitstring.Bits(float=1.23,length=32)
    print(b.bin)
    b = bitstring.Bits(float=-1.23,length=32)
    print(b.bin)
    
    a = bitstring.Bits(float=1.0,length=32)
    b = bitstring.Bits(float=-1.0,length=32)
    print(a.bin, b.bin,a.bin[:9])
    
    a = bitstring.Bits(float=1000.2090990909090909,length=32)
    b = bitstring.Bits(float=-127.0,length=32)
    print(a.bin, b.bin)
    
    a = bitstring.Bits(int=1,length=32)
    b = bitstring.Bits(int=-1,length=32)
    print(a.bin, b.bin)

In [72]:
poc_bitstring_class()

00111111100111010111000010100100
10111111100111010111000010100100
00111111100000000000000000000000 10111111100000000000000000000000 001111111
01000100011110100000110101100010 11000010111111100000000000000000
00000000000000000000000000000001 11111111111111111111111111111111


In [73]:
def poc_paillier_encryption_scheme():
    pub,priv = paillier.generate_paillier_keypair()
    n = pub.encrypt(1.25)
    m = pub.encrypt(1.25)
    print(priv.decrypt(m+n))

In [74]:
poc_paillier_encryption_scheme()

2.5


In [75]:
def poc_encryption_des():
    a = token_hex(4)
    b = token_hex(4)
    c = token_hex(4)
    
    print('Original Values : ')
    print('A : [ %s ], B : [ %s ], C : [ %s ]'%(a,b,c))
    print('--------------------------------------')
    print('First Encrypt with A and then B')
    des_in = DES.new(b, DES.MODE_ECB)
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in2.encrypt(des_in.encrypt(c))
    c_decrypted = des_in.decrypt(des_in2.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with B and then A')
    des_in = DES.new(b, DES.MODE_ECB)
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in.encrypt(des_in2.encrypt(c))
    c_decrypted = des_in2.decrypt(des_in.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with B and then B')
    des_in = DES.new(b, DES.MODE_ECB)
    c_cipher = des_in.encrypt(des_in.encrypt(c))
    c_decrypted = des_in.decrypt(des_in.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))
    
    print('--------------------------------------')
    print('First Encrypt with A and then A')
    des_in2 = DES.new(a,DES.MODE_ECB)
    c_cipher = des_in2.encrypt(des_in2.encrypt(c))
    c_decrypted = des_in2.decrypt(des_in2.decrypt(c_cipher))
    print('C Encrypted with A & B [ %s ]'%c_cipher)
    print('C Original : [ %s ], C Decrypted [ %s ]'%(c, c_decrypted))

In [76]:
poc_encryption_des()

Original Values : 
A : [ 347f48f0 ], B : [ 0da768fc ], C : [ 1f1f6723 ]
--------------------------------------
First Encrypt with A and then B
C Encrypted with A & B [ b'l\x0c\xe17\xfep\x8dx' ]
C Original : [ 1f1f6723 ], C Decrypted [ b'1f1f6723' ]
--------------------------------------
First Encrypt with B and then A
C Encrypted with A & B [ b'!\xae\x9a;8\xd4M\xa3' ]
C Original : [ 1f1f6723 ], C Decrypted [ b'1f1f6723' ]
--------------------------------------
First Encrypt with B and then B
C Encrypted with A & B [ b'\xe0@\xd9j\x89(~\xa5' ]
C Original : [ 1f1f6723 ], C Decrypted [ b'1f1f6723' ]
--------------------------------------
First Encrypt with A and then A
C Encrypted with A & B [ b'\x93\ru\xff\x80\xcb1?' ]
C Original : [ 1f1f6723 ], C Decrypted [ b'1f1f6723' ]


In [67]:
class User(object):
    def __init__(self):
        df = pd.read_csv('auto-mpg.csv')
        df = df[df['horsepower'] != '?']
        cols = df.columns.tolist()
        cols.remove('mpg')
        cols.remove('car name')
        self.X = df[cols].values.astype(np.float32)
        self.y = df['mpg'].values.astype(np.float32)
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(self.X,self.y,train_size=.8,test_size=.2)
        sc = StandardScaler()
        self.X_train = sc.fit_transform(self.X_train)
        self.X_test = sc.transform(self.X_test)
        #print(self.X_train,self.X_test)
        self.X_train = np.around(self.X_train,2)
        self.X_test = np.around(self.X_test,2)
        print(np.around(np.dot(self.X_train.T,self.X_train),2))
        self.public_key, self.private_key = paillier.generate_paillier_keypair()
        #self.paillier_encrypt = np.vectorize(public_key.encrypt)
        #self.paillier_decrypt = np.vectorize(private_key.decrypt)
    
    def paillier_encrypt(self, A_i, b_i, precision=2):
        A_i_enc = np.zeros(A_i.shape,dtype=np.object)
        for i in range(A_i.shape[0]):
            for j in range(A_i.shape[1]):
                #print(A_i[i][j])
                A_i_enc[i][j] = self.public_key.encrypt(A_i[i][j],precision=precision)
                
        b_i_enc = np.zeros(b_i.shape,dtype=np.object)
        for i in range(b_i.shape[0]):
            b_i_enc[i] = self.public_key.encrypt(b_i[i],precision=precision)
    
        
        return A_i_enc, b_i_enc
    
    def paillier_decrypt(self,c_i,precision=2):
        pass
    
    def calculate_and_send_to_evaluater(self):
        c = []
        mew_A = np.random.randn(7,7)
        mew_b = np.random.randn(7)
        #print(self.X_train.shape,self.y_train.shape)
        for x_i,y_i in zip(self.X_train,self.y_train):
            #print(x_i, y_i)
            A_i = np.dot(x_i.reshape(len(x_i), 1), x_i.reshape(len(x_i), 1).T)
            b_i = x_i * y_i
            #print(A_i.shape, b_i.shape)
            c_i = self.paillier_encrypt(A_i, b_i, precision=2)
            c.append(c_i)
        c_mew = self.paillier_encrypt(mew_A, mew_b, precision=2)
        c.append(c_mew)
        As,bs = zip(*c)
        c_final = sum(As),sum(bs)
        return c_final

In [ ]:
user = User()
c = user.calculate_and_send_to_evaluater()
for i in range(c[0].shape[0]):
    for j in range(c[0].shape[0]):
        print(user.private_key.decrypt(c[0][i][j]), end='')
        print('\t',end='')
    print()
    
for j in range(c[1].shape[0]):
        print(user.private_key.decrypt(c_final[1][j]))

In [77]:
class Evaluator(object):
    pass

In [78]:
class CSP(object):
    pass

In [79]:
class Gate(object):
    pass

class And(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        wire_labels['A_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['B_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['C_0'] = token_hex(4)
        wire_labels['C_1'] = token_hex(4)
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        for g in garbled_table:
            deciphered = des_out.decrypt(des_in.decrypt(g))
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return a, b, g, deciphered
        
        

class Or(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        wire_labels['A_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['B_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['C_0'] = token_hex(4)
        wire_labels['C_1'] = token_hex(4)
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        for g in garbled_table:
            deciphered = des_out.decrypt(des_in.decrypt(g))
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return a, b, g, deciphered

class Xor(Gate):
    def __call__(self,a,b,wire_labels={},garbled_table=[]):
        
        wire_labels['A_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['B_0'] = token_hex(4)
        wire_labels['A_1'] = token_hex(4)
        wire_labels['C_0'] = token_hex(4)
        wire_labels['C_1'] = token_hex(4)
        
        des_b_0 = DES.new(wire_labels['B_0'], DES.MODE_ECB)
        des_a_0 = DES.new(wire_labels['A_0'], DES.MODE_ECB)
        des_b_1 = DES.new(wire_labels['B_1'], DES.MODE_ECB)
        des_a_1 = DES.new(wire_labels['A_1'], DES.MODE_ECB)
        ## 0 AND 0 = 0
        
        garbled_table.append(des_a_0.encrypt(des_b_0.encrypt(wire_labels['C_0'])))
        garbled_table.append(des_a_0.encrypt(des_b_1.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_0.encrypt(wire_labels['C_1'])))
        garbled_table.append(des_a_1.encrypt(des_b_1.encrypt(wire_labels['C_0'])))
        
        des_in = DES.new(b, DES.MODE_ECB)
        des_out = DES.new(a, DES.MODE_ECB)
        for g in garbled_table:
            deciphered = des_out.decrypt(des_in.decrypt(g))
            if deciphered in [ wire_labels['C_0'], wire_labels['C_1'] ]:
                return a, b, g, deciphered
        
class Not(Gate):
    def __call__(self,inp):
        neg = {0:1,1:0}
        return neg[inp]
    
class Add(Gate):
    
    def add(self,inp1,inp2,carry):
        #xor = Xor()
        #and_g = And()
        #or_g = Or()
        o1 = Xor()(inp1,inp2)
        o2 = And()(inp1,inp2)
        add = Xor()(carry,o1)
        o4 = And()(carry,o1)
        cary = Or()(o4,o2)
        return add, cary
    
    def __call__(self,inp1,inp2):
        carry = 0
        out = np.zeros(inp1.shape)
        for idx,i in reversed(list(enumerate(zip(inp1,inp2)))):
            out[idx],carry = self.add(*i,carry)
        return out

class Subtract(Gate):
    
    def subtract(self,inp1,inp2,carry):
        #xor = Xor()
        #and_g = And()
        #or_g = Or()
        #not_g = Not()
        not_inp1 = Not()(inp1)
        o1 = Xor()(inp1, inp2)
        o2 = And()(inp2,not_inp1)
        sub = Xor()(o1, carry)
        not_o1 = Not()(o1)
        o3 = And()(carry, not_o1)
        car = Or()(o3, carry)
        return sub, car
        
    
    def __call__(self,inp1,inp2):
        carry = 0
        out = np.zeros(inp1.shape)
        for idx,i in reversed(list(enumerate(zip(inp1,inp2)))):
            out[idx], carry = self.subtract(*i,carry)
        return out

class Multiplication(Gate):

    def __call__(self,inp1,inp2):
        pass

class Division(Gate):
    
    def __call__(self,inp1,inp2):
        pass
    
class Sqrt(Gate):
    
    def __call__(self,inp):
        pass
    
class GarbledCircuit(object):
    def __init__(A,b):
        self.A = A
        self.b = b
        
    def generate_beta(self):
        pass

In [80]:
add = Add()

In [86]:
add(np.array([0,1,0]),np.array([0,1,1]))

array([1., 0., 1.])

In [82]:
Add()(np.array([0,1,0]),np.array([0,0,1]))

array([0., 1., 1.])

In [85]:
Subtract()(np.array([0,1,0]),np.array([0,0,1]))

array([0., 1., 1.])

In [101]:
bits = Bits(float=-1,length=32)
bits.bin, bits.bin[:9]

('10111111100000000000000000000000', '101111111')

In [77]:
def float_to_bin(num):
    int_p,frac_p = str(num).split('.')
    int_b = bin(int(int_p))[2:]
    frac_p = float('0.' + frac_p)
    frac_b = ''          
    for i in range(8):
        t=(frac_p*2)
        t_i,t_f = str(t).split('.')
        frac_b = frac_b + t_i
        #print(frac_p,t_i)
        if t_i == '1':
            frac_p = float(t) - 1.
        else:
            frac_p = float(t)
    return int_b.zfill(16) + '.' + frac_b

def bin_to_float(binary):
    int_p, frac_p = binary.split('.')
    integer,fractional = 0.,0. 
    for i,j in enumerate(reversed(list(int_p))):
        integer += (pow(2,i)*int(j))

    for i,j in enumerate(list(frac_p)):
        fractional += (pow(2,-(i+1))*int(j))
    
    return integer+fractional

def twos_complement(binary):
    result = ''
    for i in binary:
        result += '1' if i=='0' else '0' if i=='1' else i
    
    carry = 0
    for i in reversed(result):
        pass

In [78]:
bin_to_float(float_to_bin(1.5))

1.5

In [79]:
bin_to_float(float_to_bin(5.674))

5.671875

In [80]:
float_to_bin(1.0)

'0000000000000001.00000000'

In [82]:
'0001.1000' 

'0001.1000'

In [83]:
'1011.1000'

'1011.1000'

In [84]:
1.5 - 8.5

-7.0

In [87]:
'1101.0000'

'1101.0000'

In [85]:
'0001.0000'

'0001.0000'

In [88]:
'1111.0000'

'1111.0000'